## import library

In [1]:
import re

## class Constant
- it define the Constant

In [41]:
GAME_PLAYER = ['P1', 'P2', 'P3', 'P4', 'P5', 'P6']
CELL_MART = ['x', 'o']
TERMINAL_CONDITION = ['WIN', 'DRAW', 'LOSS']
PLAYERREWARD = {"WIN":100, "DRAW":50, "LOSS":0} 
STEPFORWARDTOEMPTY = [["?x ?y1 ?x ?y2", "?x ?y1", "?y1 ?y2", "?x ?y2", "?x1 ?y1 ?x2 ?y2", "?x1 ?y1", "?y1 ?y2", "?x1 ?x2", "?x2 ?y2", "?x1 ?y1 ?x2 ?y2", "?x1 ?y1", "?y1 ?y2", "?x2 ?x1", "?x2 ?y2"],
 ["?x ?y1 ?x ?y2", "?x ?y1", "?y2 ?y1", "?x ?y2", "?x1 ?y1 ?x2 ?y2", "?x1 ?y1", "?y2 ?y1", "?x1 ?x2", "?x2 ?y2", "?x1 ?y1 ?x2 ?y2", "?x1 ?y1", "?y2 ?y1", "?x2 ?x1", "?x2 ?y2"]]

CONNECT4COLUMN = 6



## class ReadFile
- read the Ludii file and interpret it

In [3]:
class ReadFile:
    def __init__(self, filename):
        self.GameInfo = self.loadLudii(filename)
    
    def loadLudii(self, filename):
        flagE = True
        flagR = True
        flagP = True
        
        info_dict = {}
        for var in self.pre_Process(filename):
            if not flagE:
                info_dict['equipment'] = self.ProcessEquipment(var.strip())
                flagE = True

            if not flagR:
                if var == "(":
                    count_ru += 1
                elif var == ")":
                    count_ru -=1

                if count_ru == 0 and len(string_ru) > 0:
                    list_ru.append(string_ru.strip() + ")")
                    string_ru = ""
                elif count_ru < 0:
                    info_dict['rules'] = list_ru
                    flagR = True
                else:
                    string_ru += var

            if not flagP:
                if var == "}":
                    info_dict["players"] = string_pl
                    flagP = True
                string_pl += var

            if var.strip() == 'rules':
                list_ru = []
                string_ru = ""
                flagR = False
                count_ru = 0
                pass

            if var.strip() == 'equipment':
                list_eq = []
                string_eq = ""
                count_eq = 0
                flagE = False

            elif flagE and flagR:
                if var == '(' or var == ')':
                    continue
                elif var[:4] == 'game':
                    info_dict["name"] = var.strip()[6:-1]
                elif var[:7] == 'players':
                    if var[8:] == "{":
                        string_pl = ""
                        flagP = False
                    else:
                        info_dict["players"] = var[8:]
        return info_dict
    
    def pre_Process(self, filename):
        ludCon = self.readfile(filename)
        list_pre = []
        EQU = True
        stringV = ""
        for var in ludCon:
            if not EQU:
                if var == "{":
                    if count != 0:
                        stringV += var
                    count += 1

                elif var == "}":
                    count -= 1
                    if count != 0:
                        stringV += var

                elif count == 0:
                    list_pre.append(stringV)
                    stringV = ""
                    EQU = True
                else:
                    stringV += var
            else:
                if stringV.strip() == "equipment":
                    list_pre.append(stringV.strip())
                    stringV = ""
                    count = 0
                    EQU = False

                elif var == '(' and EQU:
                    if stringV != "":
                        list_pre.append(stringV)
                        stringV = ""
                    list_pre.append('(')

                elif var == ')' and EQU:
                    list_pre.append(stringV)
                    list_pre.append(")")
                    stringV = ""

                else:
                    stringV += var
        return list_pre
    
    
    def ProcessEquipment(self, var_eq):
        list_eq = []
        string_eq = ""
        count = 0
            
        for var in var_eq:
            string_eq += var
            if var == "(":
                count += 1
            elif var == ")":
                count -= 1
            
            if count == 0 and string_eq.strip() != "":
                list_eq.append(string_eq.strip())
                string_eq = ""
        return list_eq
        
    def readfile(self, fileName):
        '''
        read the file
        :param fileName: the name of file
        return the context of file
        '''
        with open(fileName) as fp:
            content = fp.read().split("\n")
            index = 0
            for i in range(len(content)):
                content[i] = content[i].strip()
                if "//----" in content[i]:
                    index = i
                    break
            return " ".join(content[:index])
        

## class KeyTranslation
- translate the keyword of Ludii

In [4]:
class KeyTranslation:
    def __init__(self, ludii):
        self.ludii = ludii
        self.gdl = self.translate()
    
    def translate(self):
        pass


### class Game
- translate the game name
  + Ludii Format: ***game \<string>***
    * game: keyword
    * \<string>: define the name of game, it's string-based


In [5]:
class Game(KeyTranslation):
    '''
    translate the game name from ludii to GDL
    :param gameName: the name of game
    return string-based GDL
    '''
    def translate(self):
        context = ";"*80 + "\n" + f";;; {self.ludii}\n" + ";"*80 + "\n"
        return context
        

### class Players

- translate the players
  + Ludii Format: ***players \<int | string>***
    * players: keyword
    * \<int | string>: define the players
      - if using *int*, it define the size of players
      - if using *string*, it define the name of players

In [6]:
class Players(KeyTranslation):
    def translate(self):
        '''
        translate the players description from ludii to GDL
        :param players: a list that including the name of player
        return string-based GDL
        '''
        context = ";"*80 + "\n" + ";; Roles\n" + ";"*80 + "\n"
        
        if self.ludii.isdigit():
            self.num_pl = int(self.ludii)
        else:
            self.num_pl = self.StringP2DigitP()
        
        
        for index in range(self.num_pl):
            context += f"(role {GAME_PLAYER[index]})\n"
        context += "\n"
        
        context += self.translate_turn()
        return context
    
    def StringP2DigitP(self):
        count = 0
        for var in self.ludii:
            if var == "(":
                count += 1
        return count
    
    
    def translate_turn(self):
        """
        translate the mode
        :param players: the list of player
        return string-based GDL that including the turn mode
        """
        context = ";"*80 + "\n" + ";; Game Mode\n" + ";"*80 + "\n"
        for i in range(self.num_pl):
            context += f"(<= (next (control {GAME_PLAYER[i]})) (true (control {GAME_PLAYER[(i+1)%self.num_pl]})))\n"
        context += "\n"
        return context

### class Equipment
- translate equipment of ludii


In [62]:
class Equipment(KeyTranslation):
    def translate(self):
        context = ";"*80 + "\n" + ";; Initial State\n" + ";"*80 + "\n"
        self.flag_init = False
        
        for var in self.ludii:
            if var[1:6] == "board":
                self.type_board, self.size_board = self.translate_board(var[7:-1])
            
            elif var[1:6] == "piece":
                self.flag_init = self.translate_piece(var[7:-1])
                
            elif var[1:8] == "regions":
                context += self.translate_init_True(var[9:-1])
                self.flag_init = True
        
        if not self.flag_init:
            context += self.translate_init_False()
        
        else:
            context += self.translate_comparison()
        
        context += f"(init (control {GAME_PLAYER[0]}))\n\n"
        return context
            
    
    
    def translate_board(self, ludii_board):
        if re.search("\(square ([0-9]*)\)", str.lower(ludii_board)):
            return "square", int(re.search("\(square ([0-9]*)\)", str.lower(ludii_board))[1])
        
        elif re.search("\(rectangle ([0-9]* [0-9]*)\)", str.lower(ludii_board)):
            return "rectangle", int(re.search("\(rectangle ([0-9]* [0-9]*)\)", str.lower(ludii_board))[1][-1])
        return None, None
    
    def translate_piece(self, ludii_piece):
        if re.search("\"[Disc|Cross]+\" P[0-9]*", ludii_piece):
            return False
        return False
    
    # no limitation of init
    def translate_init_False(self):
        # translating square type
        context = ""
        if self.type_board == "square":
            for i in range(1, self.size_board+1):
                for j in range(1, self.size_board+1):
                    context += f"(init (cell {i} {j} b))\n"
        
        elif self.type_board == "rectangle":
            for i in range(1, self.size_board+1):
                context += f"(init (cell {i} 0 dirt))\n"
                
            for i in range(1, self.size_board+1):
                for j in range(1, CONNECT4COLUMN+1):
                    context += f"(init (cell {i} {j} b))\n"
            context += self.translate_comparison()
        return context
    
    
    def translate_init_True(self, init_ludii):
        context = ""
        player = re.search("(P[0-9]*) \(sites ([Top|Bottom]+)\)", init_ludii)[1]
        region = re.search("(P[0-9]*) \(sites ([Top|Bottom]+)\)", init_ludii)[2]
        if region == "Top":
            for j in range(1, 3):
                for i in range(1, self.size_board+1):
                    context += f"(init (cellholds {i} {j} {player}))\n"
        elif region == "Bottom":
            for j in range(2):
                for i in range(1, self.size_board+1):
                    context += f"(init (cellholds {i} {self.size_board-j} {player}))\n"
        return context
    
    def translate_comparison(self):
        context = ""
        context += "(index 0) (index 1) (index 2) (index 3) (index 4) (index 5) (index 6) (index 7) (index 8)\n" 
        context += "(++ 0 1) (++ 1 2)  (++ 2 3)  (++ 3 4)  (++ 4 5)  (++ 5 6)  (++ 6 7)  (++ 7 8)\n"
        return context

### class Rules
- translate the rules of game including(start, play, end)
  * Ludii format: ***play \<moves>***
    + play: keyword
    + \<moves>: the legal moves of playing rules

In [95]:
class Rules(KeyTranslation):
    def __init__(self, ludii, num_pl, size_board, type_board):
        self.ludii = ludii
        self.num_pl = num_pl
        self.size_board = size_board
        self.type_board = type_board
        self.gdl = self.translate()
    
    
    def translate(self):
        context = ""
        for var in self.ludii:
            if var[1:5] == "play": # should be change as keyword variable
                context += self.translate_play(var)
            
            elif var[1:4] == "end":
                context += self.translate_end(var)
        return context
    
    
    def translate_play(self, keyword_play):
        lm = LudiiMove(keyword_play[5:-1], self.num_pl)
        return lm.legalMoveGDL + lm.MoveUpdate
    
    
    def translate_end(self, keyword_end):
        le = LudiiEnd(keyword_end[5:-1], self.size_board, self.num_pl)
        return le.winCondition + le.reward + le.terminal

    

#### class LudiiMove
- translate ***move*** operator
  + add function


In [91]:
class LudiiMove:
    def __init__(self, ludii, num_pl):
        self.ludii = ludii
        self.num_pl = num_pl
        self.legalMoveGDL, self.MoveUpdate = self.translate()
    
    def translate(self):
        if self.ludii[7:10] == "Add": # should be change as keyword
            return self.translate_Add(self.ludii[11:-1])
        elif self.ludii.strip() == "(forEach Piece)": # StepForwardToEmpty
            return self.translate_StepForwardToEmpty("StepForwardToEmpty") # should be change
        return "Legal Action None\n", "Update none\n"
    
    def translate_Add(self, keyword_add):
        context_LegalAction = ";"*80 + "\n" + ";; Legal Action\n" + ";"*80 + "\n"
        context_Update = ";"*80 + "\n" + ";;  Move & Update\n" + ";"*80 + "\n"
        
        if keyword_add == "(to (sites Empty))":
            # translating legal action
            context_LegalAction += "(<= open (true (cell ?m ?n b)))\n"
            for i in range(self.num_pl):
                context_LegalAction += f"(<= (legal {GAME_PLAYER[(i+1)%self.num_pl]} noop) (true (control {GAME_PLAYER[i]})))\n"
            context_LegalAction += "(<= (legal ?w (mark ?x ?y)) (true (cell ?x ?y b)) (true (control ?w)))\n\n"
            
            # translating move update
            for i in range(self.num_pl):
                context_Update += f"(<= (next (cell ?m ?n {CELL_MART[i]})) (does {GAME_PLAYER[i]} (mark ?m ?n)) (true (cell ?m ?n b)))\n"

            # update mark cell
            context_Update += "(<= (next (cell ?m ?n ?w)) (true (cell ?m ?n ?w)) (distinct ?w b))\n"
            context_Update += "(<= (next (cell ?m ?n b)) (does ?w (mark ?j ?k)) (true (cell ?m ?n b)) (or (distinct ?m ?j) (distinct ?n ?k)))\n\n"
        
        elif "stack" in str.lower(keyword_add):
            # translating legal action
            context_LegalAction += "(<= open (true (cell ?m ?n b)))\n"
            for i in range(self.num_pl):
                context_LegalAction += f"(<= (legal {GAME_PLAYER[(i+1)%self.num_pl]} noop) (true (control {GAME_PLAYER[i]})))\n"
            context_LegalAction += "(<= (legal ?w (drop ?c)) (true (cell ?c ?y2 b)) (filled ?c ?y1) (++ ?y1 ?y2) (true (control ?w)))\n"
            
            # translating move update
            for i in range(self.num_pl):
                context_Update += f"(<= (next (cell ?c ?h2 {CELL_MART[i]})) (does {GAME_PLAYER[i]} (drop ?c)) (true (cell ?c ?h2 b)) (filled ?c ?h1) (++ ?h1 ?h2))\n"
            context_Update += "(<= (next (cell ?c ?y2 b)) (true (cell ?c ?y1 b)) (distinct ?y1 6) (++ ?y1 ?y2))\n"
            context_Update += "(<= (next (cell ?x ?y ?z)) (true (cell ?x ?y ?z)) (distinct ?z b))\n"
            context_Update += "(<= (next (cell ?c2 ?y b)) (does ?w (drop ?c1)) (true (cell ?c2 ?y b)) (distinct ?c1 ?c2))\n"
            
            for i in range(self.num_pl):
                context_Update += f"(<= (filled ?c ?h) (true (cell ?c ?h {CELL_MART[i]})))\n"
            context_Update += "(<= (filled ?c ?h) (true (cell ?c ?h dirt)))\n"
        return context_LegalAction, context_Update
    
    def translate_StepForwardToEmpty(self, keyword_forEach):
        if keyword_forEach == "StepForwardToEmpty":
            context_LegalAction = ";"*80 + "\n" + ";; Legal Action\n" + ";"*80 + "\n"
            for i in range(self.num_pl):
                context_LegalAction += f"(<= (legal {GAME_PLAYER[(i+1)%self.num_pl]} noop) (true (control {GAME_PLAYER[i]})))\n"
            
            for i in range(self.num_pl):    
                context_LegalAction += f"(<= (legal {GAME_PLAYER[i]} (move {STEPFORWARDTOEMPTY[i][0]})) (true (control {GAME_PLAYER[i]})) (true (cellholds {STEPFORWARDTOEMPTY[i][1]} {GAME_PLAYER[i]})) (++ {STEPFORWARDTOEMPTY[i][2]}) (cellempty {STEPFORWARDTOEMPTY[i][3]}))\n"
                context_LegalAction += f"(<= (legal {GAME_PLAYER[i]} (move {STEPFORWARDTOEMPTY[i][4]})) (true (control {GAME_PLAYER[i]})) (true (cellholds {STEPFORWARDTOEMPTY[i][5]} {GAME_PLAYER[i]})) (++ {STEPFORWARDTOEMPTY[i][6]}) (++ {STEPFORWARDTOEMPTY[i][7]}) (not (true (cellholds {STEPFORWARDTOEMPTY[i][8]} {GAME_PLAYER[i]}))))\n"
                context_LegalAction += f"(<= (legal {GAME_PLAYER[i]} (move {STEPFORWARDTOEMPTY[i][9]})) (true (control {GAME_PLAYER[i]})) (true (cellholds {STEPFORWARDTOEMPTY[i][10]} {GAME_PLAYER[i]})) (++ {STEPFORWARDTOEMPTY[i][11]}) (++ {STEPFORWARDTOEMPTY[i][12]}) (not (true (cellholds {STEPFORWARDTOEMPTY[i][13]} {GAME_PLAYER[i]}))))\n"
            context_LegalAction += "\n"
            
            context_Update = ";"*80 + "\n" + ";;  Move & Update\n" + ";"*80 + "\n"
            
            context_Update += "(<= (next (cellholds ?x2 ?y2 ?player)) (role ?player) (does ?player (move ?x1 ?y1 ?x2 ?y2)))\n"
            context_Update += "(<= (next (cellholds ?x3 ?y3 ?state)) (true (cellholds ?x3 ?y3 ?state)) (role ?player) (does ?player (move ?x1 ?y1 ?x2 ?y2)) (distinctcell ?x1 ?y1 ?x3 ?y3) (distinctcell ?x2 ?y2 ?x3 ?y3))\n"
            context_Update += "(<= (cell ?x ?y) (index ?x) (index ?y))\n"
            context_Update += "(<= (cellempty ?x ?y) (cell ?x ?y) (not (true (cellholds ?x ?y P1))) (not (true (cellholds ?x ?y P2))))\n"
            context_Update += "(<= (distinctcell ?x1 ?y1 ?x2 ?y2) (cell ?x1 ?y1) (cell ?x2 ?y2) (distinct ?x1 ?x2))\n"
            context_Update += "(<= (distinctcell ?x1 ?y1 ?x2 ?y2) (cell ?x1 ?y1) (cell ?x2 ?y2) (distinct ?y1 ?y2))\n\n"
            
        return context_LegalAction, context_Update
        pass
    


#### class LudiiEnd
- translate ***end*** operator
  + ludii format: ***is line \<int>***

In [162]:
class LudiiEnd:
    def __init__(self, ludii, size_board, num_pl):
        self.ludii = ludii
        self.size_board = size_board
        self.num_pl = num_pl
        self.winCondition, self.reward, self.terminal = self.translate()
    
    
    def translate(self):
        if self.ludii[1:3] == "if":
            return self.translate_if(self.ludii[4:-1])

        elif self.ludii[2:10] == "ReachWin":
            winCondition_GDL, terminal_key = LudiiWinCondition(self.ludii, self.size_board, 0, self.num_pl).translate_reachWin()
            reward_GDL = LudiiReward(terminal_key, self.num_pl).translate()
            terminal_GDL = LudiiTerminalKey(terminal_key, self.num_pl).translate()
            return winCondition_GDL, reward_GDL, terminal_GDL

        return "winCondition None\n", "Reward None\n", "Terminal None\n"
    
    
    def translate_if(self, end_if):
        list_end = self.getEnds_if(end_if)
        winCondition_GDL = ""
        reward_GDL = ""
        terminal_GDL = ""
        terminal_key = ""
        flag_draw = False
        for var in list_end:
            if re.search("\(is Line ([1-9]*)\)$", var):
                winCondition_GDL, terminal_key = LudiiWinCondition(var, self.size_board, re.search("\(is Line ([1-9]*)\)$", var)[1], self.num_pl).translate_line()
                terminal_GDL = LudiiTerminalKey(terminal_key, self.num_pl).translate()
            
            elif re.search("\(is Line ([1-9]*) byLevel:True\)$", var):
                winCondition_GDL, terminal_key = LudiiWinCondition(var, self.size_board, re.search("\(is Line ([1-9]*) byLevel:True\)$", var)[1], self.num_pl).translate_line_byLevel()
                flag_draw = True
                
            elif var[1:7] == "result":
                reward_GDL = LudiiReward(terminal_key, self.num_pl).translate_draw(flag_draw)
                terminal_GDL = LudiiTerminalKey(terminal_key, self.num_pl).translate()
                
        return winCondition_GDL, reward_GDL, terminal_GDL
                
                
    
    def getEnds_if(self, end_if):
        list_end = []
        string_end = ""
        count = 0
        
        for var in end_if:
            string_end += var
            if var == "(":
                count += 1
            
            elif var == ")":
                count -= 1
            
            if count == 0 and string_end != " ":
                list_end.append(string_end.strip())
                string_end = ""
        return list_end


    
class LudiiWinCondition:
    def __init__(self, ludii, size_board, length_line, num_pl):
        self.ludii = ludii
        self.size_board = size_board
        self.length_line = int(length_line)
        self.num_pl = num_pl
    
    def translate_line(self):
        context = ";"*80 + "\n" + ";; Win Condition\n" + ";"*80 + "\n"
        if self.size_board == self.length_line:
            contextRow = "(<= (row ?m ?x)"
            contextCol = "(<= (column ?n ?x)"
            contextDia1 = "(<= (diagonal ?x)"
            contextDia2 = "(<= (diagonal ?x)"

            for i in range(self.size_board):
                contextRow += f" (true (cell ?m {i+1} ?x))"
                contextCol += f" (true (cell {i+1} ?n ?x))"
                contextDia1 += f" (true (cell 1 {i+1} ?x))"
                contextDia2 += f" (true (cell 1 {self.size_board-i} ?x))"

            contextRow += ")\n"
            contextCol += ")\n"
            contextDia1 += ")\n"
            contextDia2 += ")\n"

            context += contextRow + contextCol + contextDia1 + contextDia2 + "\n"
            context += "(<= (line ?x) (row ?m ?x))\n(<= (line ?x) (column ?m ?x))\n(<= (line ?x) (diagonal ?x))\n\n"
            return context, "line"
    
    def translate_line_byLevel(self):
        context = ";"*80 + "\n" + ";; Win Condition\n" + ";"*80 + "\n"
        
        for i in range(1, self.length_line):
            context += f"(<= (row ?m ?x) (true (cell ?m {i} ?x)) (true (cell ?m {i+1} ?x)) (true (cell ?m {i+2} ?x)) (true (cell ?m {i+3} ?x)))\n"

        for i in range(1,self.length_line+1):
            context += f"(<= (column ?n ?x) (true (cell {i} ?n ?x)) (true (cell {i+1} ?n ?x)) (true (cell {i+2} ?n ?x)) (true (cell {i+3} ?n ?x)))\n"
        
        for j in range(1, self.length_line+1):
            for i in range(1,self.length_line):
                context += f"(<= (diagonal ?x) (true (cell {j} {i} ?x)) (true (cell {j+1} {i+1} ?x)) (true (cell {j+2} {i+2} ?x)) (true (cell {j+3} {i+3} ?x)))\n"
        
        for j in range(1, self.length_line+1):
            for i in range(1,self.length_line):
                context += f"(<= (diagonal ?x) (true (cell {j} {i+3} ?x)) (true (cell {j+1} {i+2} ?x)) (true (cell {j+2} {i+1} ?x)) (true (cell {j+3} {i} ?x)))\n"
        
        context += "(<= (line ?x) (row ?m ?x))\n(<= (line ?x) (column ?m ?x))\n(<= (line ?x) (diagonal ?x))\n\n"
        return context, "line"
    
    def translate_reachWin(self):
        context = ";"*80 + "\n" + ";; Win Condition\n" + ";"*80 + "\n"
        for i in range(self.num_pl):
            context += f"(<= {GAME_PLAYER[i]}win (index ?x) (true (cellholds ?x {(i-1)%self.size_board+1} {GAME_PLAYER[i]})))\n"
            context += f"(<= {GAME_PLAYER[i]}win (not {GAME_PLAYER[(i+1)%self.num_pl]}cell))\n"
            context += f"(<= {GAME_PLAYER[i]}cell (cell ?x ?y) (true (cellholds ?x ?y {GAME_PLAYER[i]})))\n"
        context += "\n"
        return context, "ReachWin"
        
class LudiiTerminalKey:
    def __init__(self, terminal_key, num_pl):
        self.terminal_key = terminal_key
        self.num_pl = num_pl
        
    def translate(self):
        if self.terminal_key == "line":
            context = ";"*80 + "\n" + ";;; End\n" + ";"*80 + "\n"
            for i in range(self.num_pl):
                context += f"(<= terminal(line {CELL_MART[i]}))\n"
            context += "(<= terminal(not open))\n\n" # draw
        
        elif self.terminal_key == "ReachWin":
            context = ";"*80 + "\n" + ";;; End\n" + ";"*80 + "\n"
            for i in range(self.num_pl):
                context += f"(<= terminal {GAME_PLAYER[i]}win)\n"
        return context
    

    
class LudiiReward:
    def __init__(self, reward_key, num_pl):
        self.reward_key = reward_key
        self.num_pl = num_pl
    
    
    def translate(self):
        context = ";"*80 + "\n" + ";;; Reward\n" + ";"*80 + "\n"
        if self.reward_key == "line":
            for i in range(len(TERMINAL_CONDITION)):
                for j in range(self.num_pl):
                    if PLAYERREWARD[TERMINAL_CONDITION[i]] == 100:
                        context += f"(<= (goal {GAME_PLAYER[j]} {PLAYERREWARD[TERMINAL_CONDITION[i]]}) (line {CELL_MART[j]}))\n"
                    elif PLAYERREWARD[TERMINAL_CONDITION[i]] == 0:
                        context += f"(<= (goal {GAME_PLAYER[j]} {PLAYERREWARD[TERMINAL_CONDITION[i]]}) (line {CELL_MART[j-1]}))\n"
                    else:
                        context += f"(<= (goal {GAME_PLAYER[j]} {PLAYERREWARD[TERMINAL_CONDITION[i]]}) "
                        for z in range(self.num_pl):
                            context += f"(not (line {CELL_MART[z]})) "
                        context += "(not open))\n"
        
        elif self.reward_key == "ReachWin":
            for i in range(len(TERMINAL_CONDITION)):
                for j in range(self.num_pl):
                    if PLAYERREWARD[TERMINAL_CONDITION[i]] == 100:
                        context += f"(<= (goal {GAME_PLAYER[j]} {PLAYERREWARD[TERMINAL_CONDITION[i]]}) {GAME_PLAYER[j]}win)\n"
                    elif PLAYERREWARD[TERMINAL_CONDITION[i]] == 0:
                        context += f"(<= (goal {GAME_PLAYER[j]} {PLAYERREWARD[TERMINAL_CONDITION[i]]}) {GAME_PLAYER[(j+1)%self.num_pl]}win)\n"

        return context
    
    def translate_draw(self, draw=False):
        context = self.translate()
        if draw:
            context += "(<= (goal ?w 70) (role ?w) (not (line x)) (not (line o)) open)\n"
        else:
            context += "\n"
        return context
        

### class Ludii2GDLTranslation
- translate the Ludii to GDL and write into file

In [13]:
class Ludii2GDLTranslation:
    def __init__(self, filename):
        self.filename = filename
        self.ludii = ReadFile(filename).GameInfo
    
    def convert2GDL(self):
        game_info = Game(self.ludii['name'])
        players_info = Players(self.ludii['players'])
        equipment_info = Equipment(self.ludii['equipment'])
        rules_info = Rules(a.GameInfo['rules'], players_info.num_pl, equipment_info.size_board, equipment_info.type_board)
        
        self.getGDLFile(game_info.gdl, players_info.gdl, equipment_info.gdl, rules_info.gdl)
        print("translate completed")
    
    def getGDLFile(self, game_gdl, players_gdl, equipment_gdl, rules_gdl):
        with open(self.filename.replace(".lud", ".gdl"), 'w+') as fp:
            fp.write(game_gdl)
            fp.write(players_gdl)
            fp.write(equipment_gdl)
            fp.write(rules_gdl)
        

# test

In [164]:
Ludii2GDLTranslation('ConnectFour.lud').convert2GDL()

translate completed


In [117]:
a = ReadFile('ConnectFour.lud')

#a = ReadFile('Breakthrough.lud')
print(a.GameInfo)

{'name': 'Connect Four', 'players': '2', 'equipment': ['(board (rectangle 1 7))', '(piece "Disc" Each)'], 'rules': ['(play (move Add (to (forEach (sites Board) if:(< (size Stack at:(site)) 6))) stack:True))', '(end (if (is Line 4 byLevel:True) (result Mover Win)))']}


In [118]:
a.GameInfo

{'name': 'Connect Four',
 'players': '2',
 'equipment': ['(board (rectangle 1 7))', '(piece "Disc" Each)'],
 'rules': ['(play (move Add (to (forEach (sites Board) if:(< (size Stack at:(site)) 6))) stack:True))',
  '(end (if (is Line 4 byLevel:True) (result Mover Win)))']}

In [119]:
game_info = Game(a.GameInfo['name'])
players_info = Players(a.GameInfo['players'])

print(game_info.gdl)
print(players_info.gdl)


;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;
;;; Connect Four
;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;

;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;
;; Roles
;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;
(role P1)
(role P2)

;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;
;; Game Mode
;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;
(<= (next (control P1)) (true (control P2)))
(<= (next (control P2)) (true (control P1)))




In [120]:
equipment_info = Equipment(a.GameInfo['equipment'])
print(equipment_info.gdl)

;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;
;; Initial State
;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;
(init (cell 1 0 dirt))
(init (cell 2 0 dirt))
(init (cell 3 0 dirt))
(init (cell 4 0 dirt))
(init (cell 5 0 dirt))
(init (cell 6 0 dirt))
(init (cell 7 0 dirt))
(init (cell 1 1 b))
(init (cell 1 2 b))
(init (cell 1 3 b))
(init (cell 1 4 b))
(init (cell 1 5 b))
(init (cell 1 6 b))
(init (cell 2 1 b))
(init (cell 2 2 b))
(init (cell 2 3 b))
(init (cell 2 4 b))
(init (cell 2 5 b))
(init (cell 2 6 b))
(init (cell 3 1 b))
(init (cell 3 2 b))
(init (cell 3 3 b))
(init (cell 3 4 b))
(init (cell 3 5 b))
(init (cell 3 6 b))
(init (cell 4 1 b))
(init (cell 4 2 b))
(init (cell 4 3 b))
(init (cell 4 4 b))
(init (cell 4 5 b))
(init (cell 4 6 b))
(init (cell 5 1 b))
(init (cell 5 2 b))
(init (cell 5 3 b))
(init (cell 5 4 b))
(init (cell 5 5 b))
(init (cell 5 6 b))
(init (cell 6 1 b))
(init (cell 6 2 b))
(init (cell 6 3 b))


In [121]:
a.GameInfo['rules']
Players(a.GameInfo['players']).num_pl
equipment_info.size_board
equipment_info.type_board

'rectangle'

In [163]:
#ru = Rules(a.GameInfo['rules'], Players(a.GameInfo['players']).num_pl, 3, "square")
ru = Rules(a.GameInfo['rules'], Players(a.GameInfo['players']).num_pl, equipment_info.size_board, equipment_info.type_board)
print(ru.gdl)


;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;
;; Legal Action
;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;
(<= open (true (cell ?m ?n b)))
(<= (legal P2 noop) (true (control P1)))
(<= (legal P1 noop) (true (control P2)))
(<= (legal ?w (drop ?c)) (true (cell ?c ?y2 b)) (filled ?c ?y1) (++ ?y1 ?y2) (true (control ?w)))
;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;
;;  Move & Update
;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;
(<= (next (cell ?c ?h2 x)) (does P1 (drop ?c)) (true (cell ?c ?h2 b)) (filled ?c ?h1) (++ ?h1 ?h2))
(<= (next (cell ?c ?h2 o)) (does P2 (drop ?c)) (true (cell ?c ?h2 b)) (filled ?c ?h1) (++ ?h1 ?h2))
(<= (next (cell ?c ?y2 b)) (true (cell ?c ?y1 b)) (distinct ?y1 6) (++ ?y1 ?y2))
(<= (next (cell ?x ?y ?z)) (true (cell ?x ?y ?z)) (distinct ?z b))
(<= (next (cell ?c2 ?y b)) (does ?w (drop ?c1)) (true (cell ?c2 ?y b)) (distinct ?

In [78]:
print(ru.gdl)

;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;
;;; Reward
;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;




In [24]:
a = ReadFile_test('Breakthrough.lud')
print("="*80)
print(a.GameInfo)

['(', 'game "Breakthrough" ', '(', 'players {', '(', 'player N', ')', ' ', '(', 'player S', ')', '}', ')', ' ', '(', 'equipment', ' (board (Square 8)) (piece "Pawn" Each (or { "StepForwardToEmpty" (move Step (directions {FR FL}) (to if:(or (is Empty (to)) ("IsEnemyAt" (to)) ) (apply (remove (to))) ) ) }) ) (regions P1 (sites Top)) (regions P2 (sites Bottom)) ', ' ', '(', 'rules ', '(', 'start { ', '(', 'place "Pawn1" ', '(', 'expand ', '(', 'sites Bottom', ')', '', ')', '', ')', ' ', '(', 'place "Pawn2" ', '(', 'expand ', '(', 'sites Top', ')', '', ')', '', ')', ' }', ')', '  ', '(', 'play ', '(', 'forEach Piece', ')', '', ')', '  ', '(', 'end ', '(', '"ReachWin" ', '(', 'sites Mover', ')', ' Mover', ')', '', ')', ' ', ')', ' ', ')']
{'name': 'Breakthrough', 'players': '(player N) (player S)', 'equipment': ['(board (Square 8))', '(piece "Pawn" Each (or { "StepForwardToEmpty" (move Step (directions {FR FL}) (to if:(or (is Empty (to)) ("IsEnemyAt" (to)) ) (apply (remove (to))) ) ) }) )',

In [27]:
a = ReadFile_test('ConnectFour.lud')
print(a.GameInfo)

['(', 'game "Connect Four" ', '(', 'players 2', ')', ' ', '(', 'equipment', ' (board (rectangle 1 7)) (piece "Disc" Each) ', '  ', '(', 'rules ', '(', 'play ', '(', 'move Add ', '(', 'to ', '(', 'forEach ', '(', 'sites Board', ')', ' if:', '(', '< ', '(', 'size Stack at:', '(', 'site', ')', '', ')', ' 6', ')', '', ')', '', ')', ' stack:True', ')', '', ')', ' ', '(', 'end ', '(', 'if ', '(', 'is Line 4 byLevel:True', ')', ' ', '(', 'result Mover Win', ')', '', ')', '', ')', ' ', ')', ' ', ')']
{'name': 'Connect Four', 'players': '2', 'equipment': ['(board (rectangle 1 7))', '(piece "Disc" Each)'], 'rules': ['(play (move Add (to (forEach (sites Board) if:(< (size Stack at:(site)) 6))) stack:True))', '(end (if (is Line 4 byLevel:True) (result Mover Win)))']}
